#Poetic Text Generator using RNN with LSTM cells#

##Aim##
To use LSTM neural networks (Long-Short-Term Memory) in order to teach our computer to write texts like Shakespeare. The way we do that is by training a recurrent neural network (RNN) model with LSTM cells to generate text similar to Shakespeare's writing style.

 The program you see below uses TensorFlow to download the "shakespeare.txt" file, preprocesses the text data, creates input and target sequences, defines an RNN model with LSTM cells, trains the model using the dataset, and finally generates new text using the trained model.

We import the necessary libraries:

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

We define the URL where the Shakespeare text file is located and use the requests library to download the file:

In [4]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

We know that a neural network can’t take in the raw string data, so we need to encode it by assigning numbers to each character.
We preprocess the text by creating a vocabulary of unique characters and mapping each character to an index:

We create input sequences and target sequences by splitting the text into chunks of a defined sequence length:

In [5]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

SEQ_LENGTH = 100
STEP_SIZE = 3

sentences = []
next_char = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

<ipython-input-5-6b388c12ad21>:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)
<ipython-input-5-6b388c12ad21>:19: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)


We train the model using the prepared dataset and also compile the model.
We define the architecture of the RNN model using LSTM cells:

In [ ]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=5)

Epoch 1/5
  56/1453 [>.............................] - ETA: 13:02 - loss: 3.0133

Helper function for randomnly sampling text.

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Defining a utility function to generate new and random text using the trained model:

In [ ]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

Generating texts with varying temperature parameter for its generation.

In [ ]:
print("----------0.2--------")
print(generate_text(300, 0.2))
print("----------0.4--------")
print(generate_text(300, 0.4))
print("----------0.5--------")
print(generate_text(300, 0.5))
print("----------0.6--------")
print(generate_text(300, 0.6))
print("----------0.7--------")
print(generate_text(300, 0.7))
print("----------0.8--------")
print(generate_text(300, 0.8))